**Imports and Reading Dataset**

In [3]:
import numpy as np
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

df = pd.read_csv("amazon_review.csv")


**General Informatino About Dataset**

In [4]:
def information(df):
    print("###############################    Shape  ##################################")
    print(df.shape)
    print("###############################    Types  ##################################")
    print(df.dtypes)
    print("###############################    Head   ##################################")
    print(df.head())
    print("###############################    Tail   ##################################")
    print(df.tail())
    print("###############################    NA     ##################################")
    print(df.isnull().sum())
    print("############################### Quantiles ##################################")
    print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

information(df)

###############################    Shape  ##################################
(4915, 12)
###############################    Types  ##################################
reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
day_diff            int64
helpful_yes         int64
total_vote          int64
dtype: object
###############################    Head   ##################################
       reviewerID        asin  reviewerName helpful                                         reviewText  overall                                 summary  unixReviewTime  reviewTime  day_diff  helpful_yes  total_vote
0  A3SBTW3WS4IQSN  B007WTAJTO           NaN  [0, 0]                                         No issues.   4.0000                              Four Stars      1406073600  2014-07-23       138            0           0
1 

**Calculation of Average Rating**

In [5]:
df.groupby("asin").agg({"overall": "mean"})

,overall
asin,
B007WTAJTO,4.5876


**Calculation of Weighted Average of Score by Date**

Creation of new variable by expressing the difference of each score/comment date and current_date in days and then, dividing the variable expressed in days by 4 with the quantile function and weight it according to the values from the quarters

In [15]:
df.dtypes

df["reviewTime"] = pd.to_datetime(df["reviewTime"])

current_date = df['reviewTime'].max()

df["days"] = (current_date - df["reviewTime"]).dt.days

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0000,Four Stars,1406073600,2014-07-23,138,0,0,137
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0000,nothing to really say....,1356220800,2012-12-23,715,0,0,714
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0000,best deal around,1373673600,2013-07-13,513,0,0,512


In [14]:
df['days'].quantile([.25 , .50 , .75])

0.2500   280.0000
0.5000   430.0000
0.7500   600.0000
Name: days, dtype: float64

In [17]:
print(df.loc[df["days"] <= 280, "overall"].mean())

print(df.loc[(df["days"] > 280) & (df["days"] <= 430), "overall"].mean())

print(df.loc[(df["days"] > 430) & (df["days"] <= 600), "overall"].mean())

print(df.loc[(df["days"] > 600), "overall"].mean())

4.6957928802588995
4.636140637775961
4.571661237785016
4.4462540716612375


We see that the average score has increased as the dates of the scorings get closer.
The fact that current comments are taken into account more heavily also increases the overall average.

In [7]:
def time_based_weighted_average(dataframe, q1=28, q2=26, q3=24, q4=22):
    return dataframe.loc[df["days"] <= 280, "overall"].mean() * q1 / 100 + \
           dataframe.loc[(dataframe["days"] > 280) & (dataframe["days"] <= 430), "overall"].mean() * q2 / 100 + \
           dataframe.loc[(dataframe["days"] > 430) & (dataframe["days"] <= 600), "overall"].mean() * q3 / 100 + \
           dataframe.loc[(dataframe["days"] > 600), "overall"].mean() * q4 / 100

time_based_weighted_average(df)

4.595593165128118

Average Score: 4.5876

Weighted Average Score Over Time: 4.5955

**Determination the 10 most effective reviews for the product to be displayed on the product detail page**

Generation the variable helpful_no:

• total_vote is the total number of up-downs given to a comment.

• up means helpful.

• There is no helpful_no variable in the data set, it must be generated over existing variables.

• We can find the unhelpful votes (helpful_no) by subtracting the number of useful votes (helpful_yes) from the total number of votes (total_vote).

In [18]:
df['helpful_no'] = df['total_vote'] - df['helpful_yes']

In [19]:
pos = df.iloc[:, df.columns.str.contains('helpful_yes')]

neg = df.iloc[:, df.columns.str.contains('helpful_no')]

In [20]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0000,Four Stars,1406073600,2014-07-23,138,0,0,137,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0000,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0000,best deal around,1373673600,2013-07-13,513,0,0,512,0


**Defining score_pos_neg_diff, score_average_rating and wilson_lower_bound scores**

**Up - Down Difference Score**

We calculate it by subtracting the unhelpful comments from the helpful comments

In [21]:
def score_pos_neg_diff(pos, neg):
    return pos - neg

In [27]:
df['score_pos_neg_diff'] = score_pos_neg_diff(df['helpful_yes'] , df['helpful_no'])
df.sort_values('score_pos_neg_diff' , ascending=False).head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701,68,1884,0.9663,0.9575
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578,126,1442,0.9256,0.9121
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802,77,1351,0.9488,0.9365
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032,73,349,0.8525,0.8186
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,776,27,85,0.8058,0.7321
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,806,27,55,0.7523,0.6636
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0000,I own it,1393545600,2014-02-28,283,60,68,282,8,52,0.8824,0.7847
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,157,4,41,0.9184,0.8081
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0000,Awesome Card!,1339200000,2012-09-06,823,51,65,822,14,37,0.7846,0.6703
315,A2J26NNQX6WKAU,B007WTAJTO,"Amazon Customer ""johncrea""","[38, 48]",Bought this card to use with my Samsung Galaxy...,5.0000,Samsung Galaxy Tab2 works with this card if re...,1344816000,2012-08-13,847,38,48,846,10,28,0.7917,0.6574


When we list the top 10 observations according to the up - down difference score method, it is seen that the ranking is lower in some points where the rate of useful comments is higher. For this reason, it can be misleading to rank the comments according to their benefits with this method alone.

**Score Average Rating**

We look at the ratio of the number of useful comments to all comments for that review

In [23]:
def score_average_rating(pos, neg):
    if pos + neg == 0:
        return 0
    return pos / (pos + neg)

In [28]:
df['score_average_rating'] = df.apply(lambda x: score_average_rating(x['helpful_yes'] , x['helpful_no']) , axis=1)
df.sort_values('score_average_rating' , ascending=False).head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
4277,A35KXSU6AD1481,B007WTAJTO,S. Q.,"[1, 1]",I have a galaxy note II and after rooting I no...,5.0000,Perfect!!,1355875200,2012-12-19,719,1,1,718,0,1,1.0000,0.2065
2881,A3VSG5X7GPNNW6,B007WTAJTO,Lou Thomas,"[1, 1]",The Nexus One is listed as supporting a maximu...,5.0000,Nexus One Loves This Card!,1349049600,2012-01-10,1063,1,1,1062,0,1,1.0000,0.2065
1073,A2ZXEKQ2OBZLEE,B007WTAJTO,C. Sanchez,"[1, 1]",I used it with my Samsung S4 and it works grea...,5.0000,Tons of space for phone,1376352000,2013-08-13,482,1,1,481,0,1,1.0000,0.2065
445,AIWBDRNBODLEA,B007WTAJTO,"Apache ""Elizabeth""","[1, 1]",This is exactly what I was looking for to upgr...,4.0000,Amazon Great Prices,1387324800,2013-12-18,355,1,1,354,0,1,1.0000,0.2065
3923,A2PH4RGYVR34L,B007WTAJTO,Rock Your Roots,"[1, 1]","It's a SanDisk, so what more is there to say? ...",5.0000,What more to say?,1388361600,2013-12-30,343,1,1,342,0,1,1.0000,0.2065
435,AUH8I22ITG020,B007WTAJTO,Anthony L cate,"[1, 1]",This is working great in my AT&T Galaxy Note. ...,5.0000,Love the extra storage,1343088000,2012-07-24,867,1,1,866,0,1,1.0000,0.2065
2901,A28TRYU3FJ039C,B007WTAJTO,luis,"[1, 1]",Not a good typer or speller :) here is what I ...,5.0000,Awesome and fast card :),1368403200,2013-05-13,574,1,1,573,0,1,1.0000,0.2065
2204,AANX2UN8NPE22,B007WTAJTO,"jbwam ""jbwam""","[1, 1]",I just called Sandisk and they say they have a...,2.0000,Sandisk will replace failures due to bad batch...,1371168000,2013-06-14,542,1,1,541,0,1,1.0000,0.2065
2206,A3KO3964CNP0XN,B007WTAJTO,JCBiker,"[1, 1]",I bought this for my garmin virb action cam. ...,5.0000,Great card,1383177600,2013-10-31,403,1,1,402,0,1,1.0000,0.2065
3408,A20WUUD9EDWY4N,B007WTAJTO,"Neng Vang ""Neng2012""","[1, 1]",Very good card and still working now in my car...,5.0000,working no problem,1374710400,2013-07-25,501,1,1,500,0,1,1.0000,0.2065


When we look at the 10 observations with the highest rate according to the score_average_rating method, we see that although their success rate is 100%, their frequency value is 1, which is quite misleading in terms of evaluation

**Wilson Lower Bound Score**

We observed the reasons why the Up - Down Difference Score and Score Average Rating methods can be misleading. Therefore, we will apply the Wilson Lower Bound Score method, where we can make the correct order without ignoring values such as frequency and ratio

In [25]:
def wilson_lower_bound(pos, neg, confidence=0.95):
    n = pos + neg
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * pos / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [29]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values('wilson_lower_bound' , ascending=False).head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701,68,1884,0.9663,0.9575
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802,77,1351,0.9488,0.9365
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578,126,1442,0.9256,0.9121
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032,73,349,0.8525,0.8186
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,157,4,41,0.9184,0.8081
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0000,I own it,1393545600,2014-02-28,283,60,68,282,8,52,0.8824,0.7847
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,776,27,85,0.8058,0.7321
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.0000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,648,3,19,0.8800,0.7004
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0000,Awesome Card!,1339200000,2012-09-06,823,51,65,822,14,37,0.7846,0.6703
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,806,27,55,0.7523,0.6636


When we rank according to the Wilson Lower Bound Score method, we see that the frequency and average values were taken into account and the ranking was much more accurate